## Ling Thang Midterm Version 1: Iris Dataset

## CS3210 - Machine Learning
### Instructor: Dr. Feng Jiang
### Due Date: 3/24/2024

Import the necessary libraries

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.svm as svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix , precision_score,  recall_score, auc,roc_curve,accuracy_score,f1_score
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score

# Load the iris dataset from the Pandas library

In [10]:
data = pd.read_csv('iris.csv')
df = pd.DataFrame(data)

# Displaying Raw Data and some statistics

In [11]:
# print the original dataset before encoding variety
print("\n",df.head(150))

# Generate descriptive statistics of Iris dataset
df.describe(include='all')



      sepal.length  sepal.width  petal.length  petal.width    variety
0             5.1          3.5           1.4          0.2     Setosa
1             4.9          3.0           1.4          0.2     Setosa
2             4.7          3.2           1.3          0.2     Setosa
3             4.6          3.1           1.5          0.2     Setosa
4             5.0          3.6           1.4          0.2     Setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  Virginica
146           6.3          2.5           5.0          1.9  Virginica
147           6.5          3.0           5.2          2.0  Virginica
148           6.2          3.4           5.4          2.3  Virginica
149           5.9          3.0           5.1          1.8  Virginica

[150 rows x 5 columns]


,sepal.length,sepal.width,petal.length,petal.width,variety
count,150.000000,150.000000,150.000000,150.000000,150
unique,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,Setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.843333,3.057333,3.758000,1.199333,NaN
std,0.828066,0.435866,1.765298,0.762238,NaN
min,4.300000,2.000000,1.000000,0.100000,NaN
25%,5.100000,2.800000,1.600000,0.300000,NaN
50%,5.800000,3.000000,4.350000,1.300000,NaN
75%,6.400000,3.300000,5.100000,1.800000,NaN


In [13]:
# Endcoding the variety column to numerical values
df["variety"].replace({"Setosa":0 , "Versicolor":1 , "Virginica":2}, inplace=True)
print(df.head(100))

    sepal.length  sepal.width  petal.length  petal.width  variety
0            5.1          3.5           1.4          0.2        0
1            4.9          3.0           1.4          0.2        0
2            4.7          3.2           1.3          0.2        0
3            4.6          3.1           1.5          0.2        0
4            5.0          3.6           1.4          0.2        0
..           ...          ...           ...          ...      ...
95           5.7          3.0           4.2          1.2        1
96           5.7          2.9           4.2          1.3        1
97           6.2          2.9           4.3          1.3        1
98           5.1          2.5           3.0          1.1        1
99           5.7          2.8           4.1          1.3        1

[100 rows x 5 columns]


# Data Preprocessing
#### Two separate dataframes for features and target (Variety)
X holds the features (sepal length, sepal width, petal length, petal width)
Y holds the target (Variety/Species)

### Train_Test_Split
#### 

In [28]:
#  70% for training, 10% for validation and 20% for testing
X = df.drop(columns=['variety'])
y = df['variety']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=21)

# print the shape of the training, validation and testing datasets
print("\nTraining dataset shape: ", X_train.shape, y_train.shape)  
print("Validation dataset shape: ", X_val.shape, y_val.shape)
print("Testing dataset shape: ", X_test.shape, y_test.shape)

# print the first 5 rows of the training dataset
print(pd.concat([X_train.head(), y_train.head()], axis=1))


Training dataset shape:  (105, 4) (105,)
Validation dataset shape:  (15, 4) (15,)
Testing dataset shape:  (30, 4) (30,)
     sepal.length  sepal.width  petal.length  petal.width  variety
11            4.8          3.4           1.6          0.2        0
4             5.0          3.6           1.4          0.2        0
93            5.0          2.3           3.3          1.0        1
130           7.4          2.8           6.1          1.9        2
97            6.2          2.9           4.3          1.3        1


In [31]:
def Evaluate_Performance(Model, Xtrain, Xtest, Ytrain, Ytest) : 
    Model.fit(Xtrain,Ytrain)
    overall_score = cross_val_score(Model, Xtrain,Ytrain, cv=10)
    model_score = np.average(overall_score)
    Ypredicted = Model.predict(Xtest)
    avg = 'weighted'
    print("\n • Training Accuracy Score : ", round(Model.score(Xtrain, Ytrain) * 100,2))
    print(f" • Cross Validation Score : {round(model_score * 100,2)}")
    print(f" • Testing Accuracy Score :{round(accuracy_score(Ytest, Ypredicted) * 100,2)}")
    print(f" • Precision Score is : {np.round(precision_score(Ytest, Ypredicted , average=avg) * 100,2)}")
    print(f" • Recall Score is : {np.round(recall_score(Ytest, Ypredicted , average=avg) * 100,2)}")
    print(f" • F1-Score Score is : {np.round(f1_score(Ytest, Ypredicted , average=avg) * 100,2)}")

# Logistic Regression

In [39]:
# logistic regression
logReg = LogisticRegression(solver='liblinear', multi_class='auto')
logReg.fit(X_train, y_train)
y_pred_LR = logReg.predict(X_test)
print("Logistic Regression:")
Evaluate_Performance(logReg, X_train, X_test, y_train, y_test)


Logistic Regression:

 • Training Accuracy Score :  97.14
 • Cross Validation Score : 96.09
 • Testing Accuracy Score :96.67
 • Precision Score is : 97.08
 • Recall Score is : 96.67
 • F1-Score Score is : 96.71


# Support Vector Machine

In [40]:
# Support Vector Machine 
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred_SVM = svm_model.predict(X_test)
print("\nSupport Vector Machine:")
Evaluate_Performance(svm_model, X_train, X_test, y_train, y_test)


Support Vector Machine:

 • Training Accuracy Score :  98.1
 • Cross Validation Score : 98.09
 • Testing Accuracy Score :96.67
 • Precision Score is : 97.08
 • Recall Score is : 96.67
 • F1-Score Score is : 96.71


# Decision Tree


In [41]:
DecTree = DecisionTreeClassifier(max_depth=3)
DecTree = DecTree.fit(X_train, y_train)
y_pred_DT = DecTree.predict(X_test)
print("\nDecision Tree:")
Evaluate_Performance(DecTree, X_train, X_test, y_train, y_test)


Decision Tree:

 • Training Accuracy Score :  99.05
 • Cross Validation Score : 97.09
 • Testing Accuracy Score :93.33
 • Precision Score is : 94.81
 • Recall Score is : 93.33
 • F1-Score Score is : 93.45
